# Theory

## Shrodinger's equation

The physical state of a (1-D) system for a particle is described by a (complex) wave function, $\phi(x,t)$, that follows, Shrodinger's equation:

$$
i\hbar \frac{\partial \phi(x,t)}{\partial t} = \left[-\frac{\hbar^2}{2m} \frac{\partial^2}{\partial x^2} + V(x) \right]\phi(x,t)
$$

where $V(x)$ is the potential energy of the particle at position $x$, $m$ is the mass of the particle, and $\hbar$ is the reduced Planck constant.

## Probability

The probability of finding the particle around a volume (in 1-D, a length) $dV$ around $x$ at time $t$ is given by:

$$
dP = |\phi(x,t)|^2 dV
$$

## Normalization

The wave function must be normalized, i.e., the probability of finding the particle in the entire space must be 1. This is given by:

$$
\int_{-\infty}^{\infty} |\phi(x,t)|^2 dV = 1
$$

## Change of variables

Changing variables as follows:
- Time: $t'$ $\rightarrow$ $t = t'\hbar$
- Space: $x'$ $\rightarrow$ $x = \frac{x'\hbar}{\sqrt{2m}}$

and using: 
- $\hbar = 1$
- $m = 1/2$

Shrodinger's equation becomes:

$$
-H \phi = -i \frac{\partial \phi(x,t)}{\partial t} = \left[- \frac{\partial^2}{\partial x^2} - V(x) \right]\phi(x,t)
$$

## Solution

The solution can be written in terms of eigenfunctions of the Hamiltonian operator, $H$:

$$
\phi(x,t) = \sum_{n} a_n u_{n}(x) e^{-iE_nt} + \int dk a(k) u_{k}(x) e^{-iE(k)t}
$$

where $u_{n}(x)$ are the discrete eigenfunctions of $H$ with eigenvalues $E_n$, and $u_{k}(x)$ are the continuous eigenfunctions of $H$ with eigenvalues $E(k)$.

The coefficients $a_n$ and $a(k)$ are determined by the initial conditions as:

$$
a_n = \left< u_n, \phi(x,t=0) \right>, \quad a(k) = \left< u_k, \phi(x,t=0) \right>
$$

Those eigen-functions are not always analytically solvable, and numerical methods are required.


## Discretization

The formal solution for the shrodinger equation can be written as:

$$
\phi(x,t) = e^{-i(t-t_0)H} \phi(x,t_0)
$$

Descritizing space and time as:
- Space: $x_j = jh$, $j = 0, 1, 2, \ldots, N$ (h is the space step)
- Time: $t_n = ns$, $n = 0, 1, 2, \ldots, M$ (s is the time step)

The wave function can be written as:

$$
\phi(x_j,t_n) \rightarrow \phi(jh, ns) = \phi_{j,n}
$$

Assuming boundary conditions $\phi(0,t) = \phi(Nh,t) = 0$, corresponding to a particle in a box of length $Nh$.

### Cayley approximation

The operator $e^{-isH}$ can be expressed using Cayley approximation:

$$
e^{-isH} = \frac{1 - isH_D/2}{1 + isH_D/2}
$$

The action of this operator on the wave function is:

$$
\phi_{j,n+1} = \frac{1 - isH_D/2}{1 + isH_D/2} \phi_{j,n}
$$

Rewriting this equation, we get:

$$
\phi_{j,n+1} = \left[\frac{2}{1 + isH_D/2}-1\right] \phi_{j,n} = \chi_{j,n} - \phi_{j,n}
$$

where: $\chi_{j,n} = \left[\frac{2}{1 + isH_D/2}\right] \phi_{j,n}$

Given: $\phi_{j,n}, j=0, \ldots, N; \quad \chi_{j,n}$ is the solution of the equation:

$$
[1 + isH_D/2]\chi_{j,n} = 2 \phi_{j,n}
$$

This equation can be rewritten as:

$$
\chi_{j+1,n} + \left[ -2 + \frac{2i}{\tilde{s}} - \tilde{V}_j\right]\chi_{j,n} + \chi_{j-1,n} = \frac{4i}{\tilde{s}} \phi_{j,n}
$$

where: $\tilde{s} = s/h^2$ and $\tilde{V}_j = h^2 V_j$

# Imports and setup

In [62]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp
from numba import jit

In [63]:
def potential(h, lambd, N, pos=(2/5, 3/5)):

	V = np.zeros(N, dtype=complex)
	V[int(N*pos[0]):int(N*pos[1])] = lambd*h**2

	return V

In [44]:
def alpha(s_tilda, V, N):
	
	imaginary_unit = 0 + 1j

	a = np.zeros(N, dtype=complex)

	for j in range(N-1, 0, -1):
		a[j-1] = -1 /(-2 + 2*imaginary_unit/s_tilda - V[j]+a[j])

	return a

In [64]:
@jit(nopython=True)
def betha(N, s_tilda, V, phi, a):
	
	im_unit = 0 + 1j

	b = np.zeros(N, dtype=complex)
	
	for j in range(N-1, 0, -1):
		b[j-1] = (4*im_unit*phi[j]/s_tilda - b[j])/(-2 + 2*im_unit/s_tilda - V[j] + a[j])
	return b

In [46]:
def phi_0(K, N):
	
	phi = np.zeros(N)

	for i in range(1, N):
		phi[i] = np.exp(1j*K*i)*np.exp(-8*(4*i-N)**2/N**2)

	return phi

In [65]:
@jit(nopython=True)
def wave(N, phi, X):

	for i in range(1, N):
		phi[i] = X[i] - phi[i]

	return phi

In [66]:
@jit(nopython=True)
def chi(N, alpha, betha):

	X = np.zeros(N)
	for j in range(N-1):
		X[j+1] = alpha[j]*X[j] + betha[j]

	return X

In [67]:
@jit(nopython=True)
def norm(phi, h, N):

	norm = 0

	for i in range(1, N):
		norm += np.abs(phi[i])**2*h

	return norm

In [71]:
@jit(nopython=True)
def main():

	# Some parameters
	N = int(1E3)
	n_cycles = 100
	lambd = 0.3
	h = 1
	s = 1

	t_max = int(1E4)

	# Initial conditions
	k = 2*np.pi*n_cycles/N
	s_tilda = 1

	V = potential(k, lambd, N)
	phi = phi_0(k, N)
	a = alpha(s_tilda, V, N)

	# Open files to save data
	phi_file = open("data/phi.dat", "w")
	phi_file.write("# t \t phi\n")

	norm_file = open("data/norm.dat", "w")
	norm_file.write("# t \t norm\n")


	# Time evolution
	for t in range(t_max):
		b = betha(N, s_tilda, V, phi, a)
		xi = chi(N, a, b)
		norma = norm(phi, h, N)
		
		for j in range(1, N):
			phi_file.write(f"{j},{np.abs(phi[j])}\n")
		phi = wave(N, phi, xi)

		norm_file.write(f"{t},{norma}\n")

	phi_file.close()
	norm_file.close()

In [61]:
main()

C:\Users\Denis\AppData\Local\Temp\ipykernel_13392\2397522453.py:6: ComplexWarning: Casting complex values to real discards the imaginary part
  phi[i] = np.exp(1j*K*i)*np.exp(-8*(4*i-N)**2/N**2)
C:\Users\Denis\AppData\Local\Temp\ipykernel_13392\4022710953.py:5: ComplexWarning: Casting complex values to real discards the imaginary part
  X[j+1] = alpha[j]*X[j] + betha[j]


In [72]:
# Create an animation with the data
from matplotlib.animation import FuncAnimation

def animate(i):
	data = np.loadtxt("data/phi.dat", delimiter=",")
	chunk_data = []
	for j in range(0, len(data), 1000):
		chunk_data.append(data[j])

	x = data[:,0]
	y = data[:,1]
	plt.cla()
	plt.plot(x, y)